<a href="https://colab.research.google.com/github/kotharisoumil/Info_Alchemists/blob/main/DTSC_1302_Group_20_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# Imports
import numpy as np
import pandas as pd
import io
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [36]:
url = "https://raw.githubusercontent.com/kotharisoumil/Info_Alchemists/refs/heads/main/adult.data"
column_headers = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"
]
df = pd.read_csv(url, names=column_headers)
df.head(20)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [37]:
# Count occurrences of '?' (with or without surrounding whitespace)
question_mark_count = (df.map(lambda x: str(x).strip()) == '?').sum().sum()
print("Total '?' in dataset:", question_mark_count)

Total '?' in dataset: 4262


In [38]:
# Identify columns containing '?' (with or without surrounding whitespace)
columns_with_question_marks = df.map(lambda x: str(x).strip()).eq('?').sum()

# Filter columns with at least one '?'
columns_with_question_marks = columns_with_question_marks[columns_with_question_marks > 0]
print("Columns with '?':")
print(columns_with_question_marks)

Columns with '?':
workclass         1836
occupation        1843
native_country     583
dtype: int64


In [39]:
# Remove rows with '?' in the 'occupation' column (accounting for whitespace)
df['occupation'] = df['occupation'].str.strip().replace('?', pd.NA)
df_cleaned = df.dropna(subset=['occupation'])
df_cleaned.loc[:, ["age", "occupation", "sex", "education", "income"]]

,age,occupation,sex,education,income
0,39,Adm-clerical,Male,Bachelors,<=50K
1,50,Exec-managerial,Male,Bachelors,<=50K
2,38,Handlers-cleaners,Male,HS-grad,<=50K
3,53,Handlers-cleaners,Male,11th,<=50K
4,28,Prof-specialty,Female,Bachelors,<=50K
...,...,...,...,...,...
32556,27,Tech-support,Female,Assoc-acdm,<=50K
32557,40,Machine-op-inspct,Male,HS-grad,>50K
32558,58,Adm-clerical,Female,HS-grad,<=50K
32559,22,Adm-clerical,Male,HS-grad,<=50K


In [ ]:
!git clone https://github.com/kotharisoumil/Info_Alchemists
%cd Info_Alchemists
df3 = pd.read_csv('adult.data')
df3

Cloning into 'Info_Alchemists'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 713.70 KiB | 3.41 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/Info_Alchemists/Info_Alchemists


,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
